In [7]:
import torch

pt_file = "/media/george-vengrovski/Desk SSD/BirdJEPA/llb3_val/llb3_0001_2018_04_23_14_18_03.pt"
data = torch.load(pt_file)
print(data['s'].shape)
print(data['labels'].shape)

torch.Size([128, 73])
torch.Size([73])


In [10]:


# %% birdset notebook cell (version-safe)
# ---------------------------------------------------------------------------
# pull BirdSet subsets straight into a local HF cache on your HDD
# tweak the variables below → run once per notebook session
# ---------------------------------------------------------------------------

# 1) make sure the datasets wheel is new enough for `trust_remote_code`
!pip install -q --upgrade "datasets[audio]>=2.18.0" soundfile

from importlib import reload
import datasets, pkg_resources, sys, subprocess, os
from pathlib import Path

# reload datasets in case the wheel was just upgraded in-process
reload(datasets)

# sanity check
req_ver = pkg_resources.parse_version("2.18.0")
cur_ver = pkg_resources.parse_version(datasets.__version__)
if cur_ver < req_ver:
    raise RuntimeError(
        f"datasets {cur_ver} is too old; restart the kernel or upgrade manually to >= {req_ver}"
    )

# ─── user-tunable knobs ─────────────────────────────────────────────────────
CACHE_DIR = Path("/media/george-vengrovski/Desk SSD/BirdSet")   # ⚠️ needs ~1 TB for XCL
CONFIGS   = ["XCL"]                        # e.g. ["HSN", "PER_xc", "XCL"]
SPLITS    = None                           # None → materialise all splits
NUM_PROC  = 8                              # parallel unzip/hash workers
STREAMING = False                          # True → just metadata, no audio
# ────────────────────────────────────────────────────────────────────────────

os.environ["HF_DATASETS_CACHE"] = str(CACHE_DIR.resolve())
CACHE_DIR.mkdir(parents=True, exist_ok=True)

from datasets import load_dataset

def materialise(ds, splits):
    """iterate once so the audio actually hits disk (no-op if streaming=True)."""
    for split in (splits or ds.keys()):
        for _ in ds[split]:
            pass

for cfg in CONFIGS:
    print(f"⇢ downloading BirdSet[{cfg}] to {CACHE_DIR} ...", flush=True)
    ds = load_dataset(
        "DBD-research-group/BirdSet",
        cfg,
        trust_remote_code=True,
        cache_dir=CACHE_DIR,
        num_proc=None if STREAMING else NUM_PROC,
        streaming=STREAMING,
    )
    if STREAMING:
        print("   • streaming mode: only metadata cached\n")
    else:
        materialise(ds, SPLITS)
        print("   ✓ finished materialising audio\n")



ImportError: cannot import name 'get_dataset_default_config_name' from 'datasets.inspect' (/home/george-vengrovski/anaconda3/envs/tweetybert/lib/python3.11/site-packages/datasets/inspect.py)